# Life Expectancy

Codacademy Exercise: Implementing Neural Networks

Predict life expectancy using a neural network to perform regression.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# load the dataset into a pandas DataFrame
dataset = pd.read_csv('life_expectancy.csv')

# print the first five entries in the dataset and the summary stats
print(dataset.head(5))
print(dataset.describe())

In [ ]:
# remove the country column from the dataset
dataset = dataset.drop(['country'], axis=1)

# split the data into labels and features
labels = dataset.iloc[:, -1] # select the last column
features = dataset.iloc[:, 0:-1] # select all columns except the last

# apply one-hot-encoding to the categorical columns
features = pd.get_dummies(features)

# split the data into a training set and a test set
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.20, random_state=42)

# standardize the numerical features
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
ct = ColumnTransformer([('numeric', StandardScaler(), numerical_columns)], remainder='passthrough')
features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.transform(features_test)

In [ ]:
# build the model
num_features = features.shape[1]
my_model = Sequential()
my_model.add(InputLayer(input_shape=(num_features)))
my_model.add(Dense(64, activation = 'relu')) # hidden layer
my_model.add(Dense(1)) # output layer
print(my_model.summary())

# initialize the gradient descent optimizer
opt = Adam(learning_rate = 0.01)

# compile the model
# using mean-squared error as the loss function and mean average error as the metric
my_model.compile(loss = 'mse', metrics = ['mae'], optimizer = opt)

In [ ]:
# train the model
my_model.fit(features_train_scaled, labels_train, epochs=50, batch_size=1, verbose=1)

In [ ]:
# evaluate the trained model with the test set
val_mse, val_mae = my_model.evaluate(features_test_scaled,labels_test, verbose=1)
print('MAE: ', val_mae)